<a href="https://colab.research.google.com/github/sanozzz/QuantBacktests/blob/main/Nilesh_Monthly_Rebalance(Zerodha).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
from pytz import timezone
from datetime import datetime
import logging
import re



# Define IST timezone
IST = timezone('Asia/Kolkata')

class ISTFormatter(logging.Formatter):
    """
    Custom formatter to force log timestamps to use IST timezone.
    """
    def formatTime(self, record, datefmt=None):
        # Convert the record's created time to IST
        record_time = datetime.fromtimestamp(record.created).astimezone(IST)
        if datefmt:
            return record_time.strftime(datefmt)
        return record_time.isoformat()

def setup_logger(log_file: str, level=logging.INFO, file_mode='a') -> logging.Logger:
    """
    Sets up a logger with timestamps in IST for both file and console outputs.
    Ensures no duplicate log handlers are added.

    Args:
        log_file (str): Path to the log file.
        level (int): Logging level (e.g., logging.INFO, logging.DEBUG).
        file_mode (str): Mode to open the log file ('a' for append, 'w' for overwrite).

    Returns:
        logging.Logger: Configured logger instance.
    """
    logger = logging.getLogger("OrderBookGeneratorLogger")
    logger.setLevel(level)

    # Clear existing handlers
    if logger.hasHandlers():
        logger.handlers.clear()

    # Define the custom IST formatter
    formatter = ISTFormatter('%(asctime)s - %(levelname)s - %(message)s')

    # File handler
    file_handler = logging.FileHandler(log_file, mode=file_mode)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    return logger

# Set the global timezone for the process (if on POSIX system)
os.environ['TZ'] = 'Asia/Kolkata'
try:
    time.tzset()  # Apply timezone change globally (works on POSIX systems)
except AttributeError:
    # time.tzset() is not available on non-POSIX systems (e.g., Windows)
    pass

# Define the log file path
log_file_path = f"logfile_{datetime.now(IST).strftime('%Y-%m-%d')}.log"
logger = setup_logger(log_file_path, level=logging.INFO, file_mode='a')

In [ ]:
import pandas as pd
import json
from pandas.tseries.offsets import MonthBegin

# Input and output file paths
input_file = "/content/monthly_rebalance.xlsx"
output_file = "/content/symbols.json"

# Load the Excel file
df = pd.read_excel(input_file)

# Ensure the Date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Adjust the Date to the 1st of the previous month
df['Month'] = (df['Date'] - MonthBegin(1)).dt.strftime('%Y-%m-%d')

# Build the JSON structure
cash_amount = 150000  # Fixed cash amount
result = {}
for _, row in df.iterrows():
    month = row['Month']
    symbol = row['Symbol']
    if month not in result:
        result[month] = {}
    result[month][symbol] = cash_amount

# Save the JSON to symbols.json
with open(output_file, "w") as f:
    json.dump(result, f, indent=4)

print(f"JSON file successfully created at: {output_file}")


JSON file successfully created at: /content/symbols.json


In [ ]:
import sys
sys.path.append('/content/vectorbt.pro')  # Add the path where your package is cloned
import vectorbtpro as vbt
import numpy as np
import pandas as pd
import talib
from numba import njit
import os
import warnings
from datetime import datetime, timedelta
import json
import logging
from pytz import timezone as tz
import requests
from kiteconnect import KiteConnect

# Define IST timezone
IST = tz('Asia/Kolkata')

class OrderBookGenerator:
    def __init__(self, logger, api_key, access_token, log_file, symbols_json_path='/content/symbols.json', default_cash=50000,
                 tp_ladder=[0.2,1], mult_factor=25, max_retries=5, retry_delay=2):
        """
        Initialize the OrderBookGenerator class.

        Args:
            logger: Logger object for logging.
            api_key (str): Kite API key.
            access_token (str): Kite access token.
            log_file (str): Path to the log file.
            symbols_json_path (str): Path to the JSON file containing symbol mappings.
            default_cash (int): Default initial cash value for symbols.
            tp_ladder (list): Take-profit ladder values.
            mult_factor (int): Multiplier factor for calculations.
            max_retries (int): Maximum retries for API calls.
            retry_delay (int): Delay between retries (in seconds).
        """
        self.logger = logger
        self.log_file = log_file
        self.tp_ladder = tp_ladder
        self.mult_factor = mult_factor
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.default_cash = default_cash

        # Initialize KiteConnect
        try:
            self.kite = KiteConnect(api_key=api_key)
            self.kite.set_access_token(access_token)
            self.logger.info("KiteConnect initialized successfully.")
        except Exception as e:
            self.logger.error(f"Error initializing KiteConnect: {e}")
            raise e

        # Load symbols and their respective cash values
        self.symbols_cash_map = self.load_symbols_cash(symbols_json_path)
        self.target_month = self.get_target_month()
        self.symbols_list = self.get_symbols_for_target_month()

        # Fetch instrument data from Kite API
        self.instrument_df = self.fetch_instruments()

        warnings.filterwarnings('ignore')



    def fetch_instruments(self) -> pd.DataFrame:
        """
        Fetch the list of all NSE instruments from KiteConnect API.
        """
        try:
            self.logger.info("Fetching NSE instruments from KiteConnect...")
            instrument_dump = self.kite.instruments("NSE")
            instrument_df = pd.DataFrame(instrument_dump)
            instrument_df.to_csv("NSE_Instruments.csv", index=False)  # Save as CSV for reference
            self.logger.info("Fetched and saved NSE instrument dump.")
            return instrument_df
        except Exception as e:
            self.logger.error(f"Error fetching instruments: {e}")
            raise e



    def load_symbols_cash(self, symbols_json_path: str) -> dict:
        """
        Load the JSON file mapping months to symbols with cash values.
        """
        try:
            with open(symbols_json_path, 'r') as f:
                symbols_cash_map = json.load(f)
                self.logger.info(f"Loaded symbols JSON from {symbols_json_path}")
                return symbols_cash_map
        except FileNotFoundError:
            self.logger.error(f"Symbols JSON file not found: {symbols_json_path}. Exiting.")
            raise FileNotFoundError(f"Symbols JSON file not found: {symbols_json_path}")



    def instrument_lookup(self, symbol: str) -> int:
        """
        Looks up the instrument token for a given symbol from the instrument dump.
        """
        try:
            token = self.instrument_df[self.instrument_df['tradingsymbol'] == symbol].instrument_token.values[0]
            self.logger.info(f"Found instrument token for {symbol}: {token}")
            return token
        except IndexError:
            self.logger.warning(f"Instrument token not found for symbol: {symbol}.")
            return -1

    def fetch_ohlc_extended(self, ticker: str, from_date: str, interval: str) -> pd.DataFrame:
        """
        Fetch historical data from Zerodha's KiteConnect API using extended logic to handle 100-day limit.

        Args:
            ticker (str): The trading symbol.
            from_date (str): Start date in 'YYYY-MM-DD' format.
            interval (str): Data interval (e.g., '15minute', 'day').

        Returns:
            pd.DataFrame: A DataFrame containing OHLCV data.
        """
        instrument = self.instrument_lookup(ticker)
        if instrument == -1:
            self.logger.warning(f"No instrument token found for ticker {ticker}. Skipping.")
            return pd.DataFrame()

        start_date = dt.datetime.strptime(from_date, '%Y-%m-%d')
        end_date = dt.datetime.today()

        self.logger.info(f"Fetching {interval} data for {ticker} using KiteConnect from {start_date.date()} to {end_date.date()}...")

        data = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])

        while start_date < end_date:
            fetch_end_date = min(start_date + dt.timedelta(days=100), end_date)
            try:
                data_chunk = pd.DataFrame(
                    self.kite.historical_data(instrument, start_date, fetch_end_date, interval)
                )
                data = pd.concat([data, data_chunk], ignore_index=True)
                self.logger.info(f"Fetched data from {start_date.date()} to {fetch_end_date.date()}")
            except Exception as e:
                self.logger.error(f"Error fetching data for {ticker} from {start_date.date()} to {fetch_end_date.date()}: {e}")
                break
            start_date = fetch_end_date

        if not data.empty:
            data.set_index("date", inplace=True)
            self.logger.info(f"Fetched {len(data)} rows of data for {ticker}.")
            return data[['open', 'high', 'low', 'close', 'volume']]

        self.logger.warning(f"No data found for ticker {ticker}.")
        return pd.DataFrame()

    def fetch_data(self, scrip: str, from_date: str, to_date: str, interval: str = '15minute') -> pd.DataFrame:
        """
        Wrapper for fetching historical data for a specific symbol.

        Args:
            scrip (str): The trading symbol (e.g., "RELIANCE").
            from_date (str): Start date in 'YYYY-MM-DD' format.
            to_date (str): End date in 'YYYY-MM-DD' format.
            interval (str): Data interval (e.g., '15minute', 'day').

        Returns:
            pd.DataFrame: A DataFrame containing OHLCV data.
        """
        stripped_scrip = scrip.replace("NSE:", "")
        return self.fetch_ohlc_extended(stripped_scrip, from_date, interval)






    def get_target_month(self) -> str:
            """
            Determine the target month from the JSON (format yyyy-mm-01).
            """
            today = datetime.now(IST)
            target_month = (today + timedelta(days=1)).strftime('%Y-%m-01')  # yyyy-mm-01 for next month's 1st date
            if target_month in self.symbols_cash_map:
                self.logger.info(f"Processing data for the target month: {target_month}")
                return target_month
            else:
                self.logger.error(f"No data found for the target month: {target_month}. Exiting.")
                raise ValueError(f"No data found for the target month: {target_month}")


    def get_symbols_for_target_month(self) -> list:
        """
        Extract symbols for the target month.
        """
        return list(self.symbols_cash_map[self.target_month].keys())


    @staticmethod
    @njit
    def get_final_bands_nb(close, upper, lower):
        trend = np.full(close.shape, np.nan)
        dir_ = np.full(close.shape, 1)
        long = np.full(close.shape, np.nan)
        short = np.full(close.shape, np.nan)

        for i in range(1, close.shape[0]):
            if close[i] > upper[i - 1]:
                dir_[i] = 1
            elif close[i] < lower[i - 1]:
                dir_[i] = -1
            else:
                dir_[i] = dir_[i - 1]
                if dir_[i] > 0 and lower[i] < lower[i - 1]:
                    lower[i] = lower[i - 1]
                if dir_[i] < 0 and upper[i] > upper[i - 1]:
                    upper[i] = upper[i - 1]

            if dir_[i] > 0:
                trend[i] = long[i] = lower[i]
            else:
                trend[i] = short[i] = upper[i]

        return trend, dir_, long, short

    @staticmethod
    def get_basic_bands(med_price, atr, multiplier):
        matr = multiplier * atr
        upper = med_price + matr
        lower = med_price - matr
        return upper, lower

    def faster_supertrend_talib(self, high, low, close, period=7, multiplier=3):
        avg_price = talib.MEDPRICE(high.flatten(), low.flatten())
        atr = talib.ATR(high.flatten(), low.flatten(), close.flatten(), period)
        upper, lower = self.get_basic_bands(avg_price, atr, multiplier)
        return self.get_final_bands_nb(close, upper, lower)


    @staticmethod
    def shift_array(arr: np.ndarray) -> np.ndarray:
        shifted = np.zeros_like(arr)
        shifted[1:] = arr[:-1]
        return shifted



    def MTF_Trend(self, high: pd.Series, low: pd.Series, close: pd.Series) -> np.ndarray:
        # Resample to weekly and daily closes
        close_1W = close.resample("1w").last()
        close_1D = close.resample("1d").last().dropna().shift(1)

        # Resample weekly and daily data to 15m
        resampler_w = vbt.Resampler(close_1W.index, close.index, source_freq="W-SUN", target_freq="15m")
        resampler_d = vbt.Resampler(close_1D.index, close.index, source_freq="1d", target_freq="15m")

        # Compute multiple SuperTrend indicators
        _, superd_1, _, _ = self.faster_supertrend_talib(high.values, low.values, close.values, 7, 3)
        _, superd_2, _, _ = self.faster_supertrend_talib(high.values, low.values, close.values, 10, 3)
        _, superd_3, _, _ = self.faster_supertrend_talib(high.values, low.values, close.values, 11, 2)

        # Combine SuperTrend directions
        concatenated = superd_1 + superd_2 + superd_3
        shifted = self.shift_array(concatenated)

        # Determine strong trend conditions
        condition_3 = (concatenated == 3) & (shifted != 3)
        condition_neg_3 = (concatenated == -3) & (shifted != -3)

        # Define short-term trend
        ST_Trend = np.where(condition_3, 1, 0)
        ST_Trend = np.where(condition_neg_3, -1, ST_Trend)

        # Weekly SMA and MACD-based conditions
        SMA_1W = vbt.talib("SMA").run(close_1W, timeperiod=20).real
        MACD = vbt.talib_func("MACD")
        MACD_Hist = MACD(close_1W, 12, 26, 9)[2]
        MACD_Positive = MACD_Hist > 0

        Cls1W_Abv_SMA = close_1W.vbt >= SMA_1W

        # Daily SMA alignment
        SMA20_1D = vbt.talib("SMA").run(close_1D, timeperiod=20).real
        Close15mBelow20DMA = close.vbt <= vbt.talib("SMA").run(close_1D, timeperiod=20).real.vbt.realign(resampler_d)

        # Long-term trend (weekly SMA and MACD alignment)
        LT_Trend = np.where(
            np.logical_and(
                MACD_Positive.vbt.realign(resampler_w).to_numpy(),
                Cls1W_Abv_SMA.vbt.realign(resampler_w).to_numpy()
            ),
            1,
            0
        )

        # Combine short-term and long-term trends
        MTF_Trend = np.where((LT_Trend == 1) & (ST_Trend == 1), 1, 0)
        MTF_Trend = np.where(Close15mBelow20DMA.to_numpy(), -1, MTF_Trend)

        return MTF_Trend


    def fetch_data(self, scrip: str, from_date: str, to_date: str, interval: str = '15minute') -> pd.DataFrame:
        """
        Fetch historical candle data using KiteConnect API.

        Args:
            scrip (str): The trading symbol to fetch data for.
            from_date (str): Start date in 'YYYY-MM-DD' format.
            to_date (str): End date in 'YYYY-MM-DD' format.
            interval (str): Timeframe for the candles. Default is '15minute'.

        Returns:
            pd.DataFrame: DataFrame with OHLCV data.
        """
        try:
            # Strip "NSE:" prefix if present
            stripped_scrip = scrip.replace("NSE:", "")

            # Retrieve the instrument token
            instrument_token = self.instrument_lookup(stripped_scrip)
            if instrument_token == -1:
                self.logger.warning(f"Instrument token not found for scrip: {scrip}. Skipping.")
                return None

            self.logger.info(f"Fetching {interval} data for {stripped_scrip} using KiteConnect from {from_date} to {to_date}...")

            start_date = datetime.strptime(from_date, '%Y-%m-%d')
            end_date = datetime.strptime(to_date, '%Y-%m-%d')
            data = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])

            # Fetch data in chunks to handle API limitations
            while start_date < end_date:
                chunk_end_date = min(start_date + timedelta(days=100), end_date)
                try:
                    chunk = pd.DataFrame(
                        self.kite.historical_data(
                            instrument_token,
                            from_date=start_date,
                            to_date=chunk_end_date,
                            interval='15minute' if interval == '15minute' else interval
                        )
                    )
                    data = pd.concat([data, chunk], ignore_index=True)
                    self.logger.info(f"Fetched data from {start_date.date()} to {chunk_end_date.date()} for {stripped_scrip}.")
                except Exception as e:
                    self.logger.error(f"Error fetching data for {scrip} from {start_date.date()} to {chunk_end_date.date()}: {e}")
                    break
                start_date = chunk_end_date

            if data.empty:
                self.logger.warning(f"No data found for scrip: {scrip}.")
                return None

            # Post-process the data
            data['date'] = pd.to_datetime(data['date'])
            data.set_index('date', inplace=True)

            self.logger.info(f"Data fetched successfully for {scrip}. Total rows: {len(data)}.")
            return data[['open', 'high', 'low', 'close', 'volume']]
        except Exception as e:
            self.logger.error(f"Error fetching data for {scrip}: {e}")
            return None



    def OrderGenerator(self, scrip: str, from_date: str, to_date: str):
        """
        Generate orders for a specific scrip within the given date range.

        Args:
            scrip (str): The symbol for which to generate orders.
            from_date (str): Start date in 'YYYY-MM-DD' format.
            to_date (str): End date in 'YYYY-MM-DD' format.

        Returns:
            pd.DataFrame or None: A DataFrame of trades if successful, else None.
        """
        # Retrieve `init_cash` value for the current scrip and target month
        init_cash = self.symbols_cash_map.get(self.target_month, {}).get(scrip, self.default_cash)
        self.logger.info(f"Processing {scrip} for the target month {self.target_month} with init_cash={init_cash}.")

        # Fetch data for the specified range
        # Ensure `to_date` does not exceed today's date
        today = datetime.now(IST).strftime('%Y-%m-%d')
        to_date = min(to_date, today)

        data = self.fetch_data(scrip, from_date, to_date, interval='15minute')

        if data is None or data.empty:
            self.logger.warning(f"No data available for {scrip} during {from_date} to {to_date}. Skipping.")
            return None

        self.logger.info(f"Data successfully fetched for {scrip} with {len(data)} rows.")

        # Ensure required columns exist
        if not {'open', 'high', 'low', 'close'}.issubset(data.columns):
            self.logger.error(f"Data for {scrip} is missing required columns. Skipping.")
            return None

        high, low, close = data['high'], data['low'], data['close']

        # Generate signals using SuperTrend
        Signals = vbt.IF(
            class_name='SuperTrend',
            short_name='st',
            input_names=['high', 'low', 'close'],
            param_names=[],
            output_names=['value']
        ).with_apply_func(
            self.MTF_Trend, keep_pd=True
        ).run(high, low, close)

        # Create portfolio based on generated signals
        pf = vbt.Portfolio.from_signals(
            data,
            long_entries=(Signals.value == 1),
            long_exits=(Signals.value == -1),
            stop_exit_price="close",
            sl_stop=data['low'].vbt.ago(125),
            stop_ladder="uniform",
            tp_stop=vbt.Param([self.tp_ladder], keys=["tp_ladder_1"]),
            tsl_stop=0.2,
            fees=0.0015,  # 0.15% of turnover
            slippage=0.0005,  # 0.05% slippage
            size_granularity=1,
            max_size=init_cash,
            init_cash=init_cash,
            freq='15m'
        )

        # Extract readable trades data
        df = pf.trades.readable
        df["Column"] = scrip

        # Validate the DataFrame's structure
        if df.empty or 'Entry Index' not in df.columns:
            self.logger.warning(f"No trades generated or 'Entry Index' missing for {scrip}.")
            return None

        self.logger.info(f"Processing completed for {scrip} in target month {self.target_month}.")
        return df


    def calculate_days_difference(self):
        """
        Calculate the number of days from the target month's start to today, multiplied by a factor.
        """
        # Ensure the current date is timezone-aware
        today_date = datetime.now(IST)

        # Convert target_month to a timezone-aware datetime
        target_date = datetime.strptime(self.target_month, "%Y-%m-%d").replace(tzinfo=IST)

        # Perform subtraction and calculate the difference in days
        difference_days = (today_date - target_date).days

        return difference_days * self.mult_factor


    def exponential_backoff(self, function, *args, **kwargs):
        retries = 0
        delay = self.retry_delay
        while retries < self.max_retries:
            try:
                return function(*args, **kwargs)
            except Exception as e:
                retries += 1
                self.logger.error(f"Attempt {retries} failed with error: {e}")
                if retries >= self.max_retries:
                    raise e
                time.sleep(delay)
                delay *= 2



    def get_orderbook(self, max_months=12):
        """
        Generate the order book by processing the latest `max_months` months in the JSON.
        Runs OrderGenerator separately for each symbol for each target month.
        """
        final_results = []  # List to store results for all months

        # Get sorted months from the JSON keys (latest first)
        sorted_months = sorted(self.symbols_cash_map.keys(), key=lambda x: datetime.strptime(x, "%Y-%m-%d"), reverse=True)
        sorted_months = sorted_months[:max_months]  # Process only the first `max_months` months

        # Process each target month
        for target_month in sorted_months:
            self.logger.info(f"Processing data for the target month: {target_month}")
            self.target_month = target_month  # Set the target month for the current loop

            month_start = IST.localize(datetime.strptime(target_month, "%Y-%m-%d"))
            from_date = (month_start - timedelta(days=365)).strftime('%Y-%m-%d')  # Start 365 days before the target month
            to_date = (month_start + timedelta(days=90)).strftime('%Y-%m-%d')  # End 90 days after the target month

            # Ensure the `to_date` does not exceed today's date
            today = datetime.now(IST).strftime('%Y-%m-%d')
            to_date = min(to_date, today)

            # Process each symbol for the target month
            if target_month not in self.symbols_cash_map:
                self.logger.warning(f"No data found for the target month: {target_month}. Skipping.")
                continue

            symbols = self.symbols_cash_map[target_month].keys()
            dfs = []
            for symbol in symbols:
                self.logger.info(f"Running OrderGenerator for symbol: {symbol} in month: {target_month}.")

                # Fetch data and generate orders
                df = self.OrderGenerator(symbol, from_date=from_date, to_date=to_date)
                if df is not None and not df.empty:
                    # Filter rows for the target month
                    filtered_df = df[
                        (df['Entry Index'] >= month_start) &
                        (df['Entry Index'] < month_start + timedelta(days=31))
                    ]

                    if not filtered_df.empty:
                        dfs.append(filtered_df)

            if dfs:
                # Combine all symbols' dataframes for the current month
                final_results.append(pd.concat(dfs, ignore_index=True))
            else:
                self.logger.warning(f"No valid orders found for target month: {target_month}. Skipping.")

        if not final_results:
            self.logger.info("No valid orders found for the specified months. Returning an empty orderbook.")
            return pd.DataFrame()

        # Combine results for all processed months into a single DataFrame
        final_orderbook = pd.concat(final_results, ignore_index=True)

        # Log summary instead of full data
        self.logger.info(f"Order book generated successfully! Total trades: {len(final_orderbook)}")
        return final_orderbook










In [ ]:
log_file_path = f"logfile_{datetime.now(IST).strftime('%Y-%m-%d')}.log"

# Setup logger
logger = logging.getLogger("OrderBookGeneratorLogger")
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
logger.addHandler(console_handler)



api_key = "lwhfs47ilbvwc84g"
access_token = "ClBkOq5DY2jVdd729bWl2psI6iMQr0Fy"

# Instantiate the class
OBGenerator = OrderBookGenerator(
    logger=logger,
    log_file=log_file_path,
    symbols_json_path='/content/symbols.json',
    api_key = api_key,
    access_token=access_token
)

# Test if get_orderbook is recognized
if hasattr(OBGenerator, 'get_orderbook'):
    print("get_orderbook method exists!")
else:
    print("get_orderbook method is missing!")

# Generate the order book
try:
    df = OBGenerator.get_orderbook(max_months=120)
    print("Order book generated successfully!")
    print(df.head())
except AttributeError as e:
    print(f"AttributeError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


# Ensure datetime columns are properly formatted
df["Entry Index"] = pd.to_datetime(df["Entry Index"])
df["Exit Index"] = pd.to_datetime(df["Exit Index"], errors='coerce')  # Allow for NaT for open trades

# The final DataFrame `df` contains the complete order book for the latest 12 months
df.to_csv('tradebook.csv')


2024-12-01T19:41:47.330129+05:30 - INFO - KiteConnect initialized successfully.
KiteConnect initialized successfully.
INFO:OrderBookGeneratorLogger:KiteConnect initialized successfully.
2024-12-01T19:41:47.342374+05:30 - INFO - Loaded symbols JSON from /content/symbols.json
Loaded symbols JSON from /content/symbols.json
INFO:OrderBookGeneratorLogger:Loaded symbols JSON from /content/symbols.json
2024-12-01T19:41:47.351542+05:30 - INFO - Processing data for the target month: 2024-12-01
Processing data for the target month: 2024-12-01
INFO:OrderBookGeneratorLogger:Processing data for the target month: 2024-12-01
2024-12-01T19:41:47.355837+05:30 - INFO - Fetching NSE instruments from KiteConnect...
Fetching NSE instruments from KiteConnect...
INFO:OrderBookGeneratorLogger:Fetching NSE instruments from KiteConnect...
2024-12-01T19:41:47.898199+05:30 - INFO - Fetched and saved NSE instrument dump.
Fetched and saved NSE instrument dump.
INFO:OrderBookGeneratorLogger:Fetched and saved NSE ins

get_orderbook method exists!


Streaming output truncated to the last 5000 lines.
Fetched data from 2015-06-10 to 2015-09-18 for VEEDOL.
INFO:OrderBookGeneratorLogger:Fetched data from 2015-06-10 to 2015-09-18 for VEEDOL.
2024-12-01T20:25:31.380772+05:30 - INFO - Fetched data from 2015-09-18 to 2015-12-27 for VEEDOL.
Fetched data from 2015-09-18 to 2015-12-27 for VEEDOL.
INFO:OrderBookGeneratorLogger:Fetched data from 2015-09-18 to 2015-12-27 for VEEDOL.
2024-12-01T20:25:31.893052+05:30 - INFO - Fetched data from 2015-12-27 to 2016-04-05 for VEEDOL.
Fetched data from 2015-12-27 to 2016-04-05 for VEEDOL.
INFO:OrderBookGeneratorLogger:Fetched data from 2015-12-27 to 2016-04-05 for VEEDOL.
2024-12-01T20:25:32.454542+05:30 - INFO - Fetched data from 2016-04-05 to 2016-05-30 for VEEDOL.
Fetched data from 2016-04-05 to 2016-05-30 for VEEDOL.
INFO:OrderBookGeneratorLogger:Fetched data from 2016-04-05 to 2016-05-30 for VEEDOL.
2024-12-01T20:25:32.475549+05:30 - INFO - Data fetched successfully for NSE:VEEDOL. Total rows: 76

Order book generated successfully!
   Exit Trade Id        Column   Size  Entry Order Id  \
0             12  NSE:AEROFLEX  714.0              23   
1             13  NSE:AEROFLEX  699.0              25   
2             13      NSE:NSIL   13.0              24   
3             14      NSE:NSIL   14.0              24   
4             15      NSE:NSIL   26.0              27   

                Entry Index  Avg Entry Price  Entry Fees  Exit Order Id  \
0 2024-11-01 18:00:00+05:30       195.477690  209.356606             24   
1 2024-11-05 09:15:00+05:30       194.046975  203.458253             26   
2 2024-11-06 13:00:00+05:30      6427.212000  125.330634             25   
3 2024-11-06 13:00:00+05:30      6427.212000  134.971452             26   
4 2024-11-22 09:15:00+05:30      9309.652500  363.076448             28   

                 Exit Index  Avg Exit Price   Exit Fees           PnL  \
0 2024-11-04 09:15:00+05:30      190.514695  204.041238  -3956.976274   
1 2024-11-05 09:30:00+05:

In [ ]:
import pandas as pd
import json
import numpy as np
import vectorbtpro as vbt
from datetime import datetime
import logging


class TradeSummaryGenerator:
    def __init__(self, logger):
        """
        Initialize the TradeSummaryGenerator with a logger.
        """
        self.logger = logger

    def fetch_equal_weighted_returns_monthly(self, symbols, month):
        """
        Fetch monthly candle data for the given symbols and calculate equal-weighted returns.
        """
        try:
            self.logger.info(f"Fetching equal-weighted returns for month: {month}")
            monthly_returns = []

            if not symbols:
                self.logger.warning(f"No symbols provided for month: {month}")
                return np.nan

            for symbol in symbols:
                try:
                    # Fetch monthly data
                    self.logger.info(f"Pulling data for symbol: {symbol}")
                    data = vbt.TVData.pull(symbol, timeframe="1M", tz="GMT+5:30")
                    close = data.get("Close")

                    # Ensure timezone-awareness
                    if close.index.tz is None:
                        close = close.tz_localize("UTC").tz_convert("Asia/Kolkata")
                    else:
                        close = close.tz_convert("Asia/Kolkata")

                    # Get Close prices for the current and previous months
                    current_month = pd.Timestamp(f"{month}-01", tz="Asia/Kolkata")
                    previous_month = (current_month - pd.offsets.MonthBegin(1))

                    # Get Close prices for the two months
                    previous_month_close = close.loc[close.index.to_period('M') == previous_month.to_period('M')]
                    current_month_close = close.loc[close.index.to_period('M') == current_month.to_period('M')]

                    if not previous_month_close.empty and not current_month_close.empty:
                        return_value = current_month_close.iloc[-1] / previous_month_close.iloc[-1] - 1
                        monthly_returns.append(return_value)
                        self.logger.info(f"Return for symbol {symbol}: {return_value:.4f}")
                    else:
                        self.logger.warning(f"Skipping symbol due to insufficient data: {symbol}, month: {month}")

                except Exception as e:
                    self.logger.error(f"Error fetching data for symbol: {symbol}, error: {e}")

            # Calculate equal-weighted return
            if monthly_returns:
                result = round(np.mean(monthly_returns), 4)
                self.logger.info(f"Equal-weighted return for month {month}: {result}")
                return result
            else:
                self.logger.warning(f"No valid returns calculated for month: {month}")
                return np.nan
        except Exception as e:
            self.logger.error(f"Error fetching equal-weighted returns for month: {month}, error: {e}")
            return np.nan

    def fetch_cnxsmallcap_returns(self):
        """
        Fetch monthly adjusted close prices for CNXSMALLCAP using vbt.TVData.pull and calculate monthly returns.
        """
        try:
            self.logger.info("Fetching CNXSMALLCAP returns...")
            # Fetch data for CNXSMALLCAP
            cnxsmallcap_data = vbt.TVData.pull(
                "NSE:CNXSMALLCAP",  # Correct symbol for CNXSMALLCAP
                timeframe="1M",  # Monthly timeframe
                tz="GMT+5:30"
            )
            # Extract adjusted close prices
            cnxsmallcap_close = cnxsmallcap_data.get("Close")

            # Calculate monthly returns
            cnxsmallcap_returns = cnxsmallcap_close.pct_change().rename("CNXSMALLCAP_Returns").to_frame()

            # Round CNXSMALLCAP returns to 2 decimals
            cnxsmallcap_returns['CNXSMALLCAP_Returns'] = cnxsmallcap_returns['CNXSMALLCAP_Returns'].round(2)

            # Add a 'Month' column
            cnxsmallcap_returns['Month'] = cnxsmallcap_returns.index.to_period('M')
            self.logger.info("CNXSMALLCAP returns fetched successfully.")
            return cnxsmallcap_returns
        except Exception as e:
            self.logger.error(f"Error fetching CNXSMALLCAP data: {e}")
            return pd.DataFrame()


    def calculate_metrics(self, group):
        """
        Calculate additional metrics for the tradebook summary.
        """
        self.logger.info("Calculating metrics for a trade group...")
        winning_trades = group[group['PnL'] > 0]
        losing_trades = group[group['PnL'] < 0]

        metrics = {
            'Month_PnL': round(group['PnL'].sum(), 2),
            'Month_Trades': len(group),
            'Unique_Scrips': group['Column'].nunique(),
            'Max_Loss': round(group['PnL'].min(), 2),
            'Max_Profit': round(group['PnL'].max(), 2),
            'Median_Profit': round(winning_trades['PnL'].median(), 2) if not winning_trades.empty else 0,
            'Median_Loss': round(losing_trades['PnL'].median(), 2) if not losing_trades.empty else 0,
            'Win_Rate': round(len(winning_trades) / len(group), 2) if len(group) > 0 else 0,
            'Profit_Factor': round(winning_trades['PnL'].sum() / abs(losing_trades['PnL'].sum()), 2) if not losing_trades.empty else float('inf'),
            'R_Multiple': round((winning_trades['Return'].mean() / abs(losing_trades['Return'].mean())), 2) if not losing_trades.empty else float('inf'),
        }
        self.logger.info(f"Metrics calculated: {metrics}")
        return pd.Series(metrics)

    def generate_summary(self, tradebook_df, symbols_data, cnxsmallcap_returns):
        """
        Generate a summary of trade metrics and merge with CNXSMALLCAP returns and cumulative equal-weighted returns.
        """
        self.logger.info("Generating summary from the tradebook...")

        # Ensure 'Month' column exists
        if 'Month' not in tradebook_df.columns:
            if 'Entry Index' in tradebook_df.columns:
                self.logger.info("Creating 'Month' column from 'Entry Index'.")
                tradebook_df['Entry Index'] = pd.to_datetime(tradebook_df['Entry Index'], errors='coerce')
                tradebook_df['Month'] = tradebook_df['Entry Index'].dt.to_period('M')  # Extract month as Period[M]
            else:
                self.logger.error("Missing required 'Entry Index' column to create 'Month'.")
                raise KeyError("The 'Month' column is missing, and 'Entry Index' is not available to create it.")

        # Group by 'Month' and apply metrics calculation
        summary = tradebook_df.groupby('Month').apply(self.calculate_metrics).reset_index()

        # Add number of scrips from the symbols.json
        summary['Num_Scrips_Input'] = summary['Month'].apply(
            lambda x: len(symbols_data.get(str(x) + "-01", {}))
        )

        # Add strategy return early
        summary['Strategy_Return'] = round(summary['Month_PnL'] / 1_000_000, 4)

        # Calculate equal-weighted returns (without cost adjustment)
        summary['Equal_Weighted_Return'] = summary['Month'].apply(
            lambda month: self.fetch_equal_weighted_returns_monthly(symbols_data.get(str(month) + "-01", {}).keys(), str(month))
        )

        # Calculate equal-weighted returns (cost-adjusted by 0.4%)
        summary['Equal_Weighted_Return_Cost_Adjusted'] = summary['Equal_Weighted_Return'] - 0.004

        # Calculate cumulative equal-weighted returns
        summary['Cumulative_Equal_Weighted_Return'] = round((1 + summary['Equal_Weighted_Return']).cumprod() - 1, 4)

        # Calculate cumulative equal-weighted returns (cost-adjusted by 0.4%)
        summary['Cumulative_Equal_Weighted_Return_Cost_Adjusted'] = round((1 + summary['Equal_Weighted_Return_Cost_Adjusted']).cumprod() - 1, 4)

        # Add cumulative returns with reinvestments
        summary['Strategy_Cumulative_Return_With_Reinvestments'] = round(
            (1 + summary['Strategy_Return']).cumprod() - 1, 4
        )

        summary['Equal_Weighted_Return_Cost_Adjusted_With_Reinvestments'] = round(
            (1 + summary['Equal_Weighted_Return_Cost_Adjusted']).cumprod() - 1, 4
        )

        # Merge the CNXSMALLCAP monthly returns into the summary
        summary = summary.merge(cnxsmallcap_returns, on='Month', how='left')

        # Calculate cumulative returns for both CNXSMALLCAP and the strategy
        summary['Cumulative_Strategy_Return'] = round((1 + summary['Strategy_Return']).cumprod() - 1, 4)
        summary['Cumulative_CNXSMALLCAP_Return'] = round((1 + summary['CNXSMALLCAP_Returns']).cumprod() - 1, 4)

        # Calculate drawdowns
        summary['Max_Cumulative_Strategy_Return'] = summary['Cumulative_Strategy_Return'].cummax()
        summary['Strategy_Drawdown'] = round((summary['Cumulative_Strategy_Return'] - summary['Max_Cumulative_Strategy_Return']) / summary['Max_Cumulative_Strategy_Return'], 4)

        summary['Max_Cumulative_CNXSMALLCAP_Return'] = summary['Cumulative_CNXSMALLCAP_Return'].cummax()
        summary['CNXSMALLCAP_Drawdown'] = round((summary['Cumulative_CNXSMALLCAP_Return'] - summary['Max_Cumulative_CNXSMALLCAP_Return']) / summary['Max_Cumulative_CNXSMALLCAP_Return'], 4)

        summary['Max_Cumulative_Equal_Weighted_Return'] = summary['Cumulative_Equal_Weighted_Return'].cummax()
        summary['Equal_Weighted_Drawdown'] = round((summary['Cumulative_Equal_Weighted_Return'] - summary['Max_Cumulative_Equal_Weighted_Return']) / summary['Max_Cumulative_Equal_Weighted_Return'], 4)

        summary['Max_Cumulative_Equal_Weighted_Return_Cost_Adjusted'] = summary['Cumulative_Equal_Weighted_Return_Cost_Adjusted'].cummax()
        summary['Equal_Weighted_Drawdown_Cost_Adjusted'] = round((summary['Cumulative_Equal_Weighted_Return_Cost_Adjusted'] - summary['Max_Cumulative_Equal_Weighted_Return_Cost_Adjusted']) / summary['Max_Cumulative_Equal_Weighted_Return_Cost_Adjusted'], 4)

        # Calculate current drawdowns
        current_drawdowns = {
            'Current_Strategy_Drawdown': summary['Strategy_Drawdown'].iloc[-1],
            'Current_CNXSMALLCAP_Drawdown': summary['CNXSMALLCAP_Drawdown'].iloc[-1],
            'Current_Equal_Weighted_Drawdown': summary['Equal_Weighted_Drawdown'].iloc[-1],
            'Current_Equal_Weighted_Drawdown_Cost_Adjusted': summary['Equal_Weighted_Drawdown_Cost_Adjusted'].iloc[-1]
        }

        for metric, value in current_drawdowns.items():
            self.logger.info(f"{metric}: {value:.4%}")

        # Drop unnecessary columns
        summary = summary.drop(columns=[
            'Max_Cumulative_Strategy_Return',
            'Max_Cumulative_CNXSMALLCAP_Return',
            'Max_Cumulative_Equal_Weighted_Return',
            'Max_Cumulative_Equal_Weighted_Return_Cost_Adjusted'
        ], errors='ignore')

        self.logger.info("Summary generated successfully.")
        return summary



In [ ]:
# Set up the logger
log_file_path = "trade_summary.log"
logging.basicConfig(level=logging.INFO, filename=log_file_path, filemode="w", format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

# Create an instance of the class
summary_generator = TradeSummaryGenerator(logger)

# Example inputs
tradebook_df = pd.read_csv("tradebook.csv")  # Load the tradebook data
symbols_data = json.load(open("symbols.json"))  # Load the symbols.json
cnxsmallcap_returns = summary_generator.fetch_cnxsmallcap_returns()

# Generate the summary
summary = summary_generator.generate_summary(tradebook_df, symbols_data, cnxsmallcap_returns)

# Save the summary to a CSV
summary.to_csv("summary.csv", index=False)
logger.info("Summary saved to summary.csv.")


In [ ]:
summary

,Month,Month_PnL,Month_Trades,Unique_Scrips,Max_Loss,Max_Profit,Median_Profit,Median_Loss,Win_Rate,Profit_Factor,...,Cumulative_Equal_Weighted_Return_Cost_Adjusted,Strategy_Cumulative_Return_With_Reinvestments,Equal_Weighted_Return_Cost_Adjusted_With_Reinvestments,CNXSMALLCAP_Returns,Cumulative_Strategy_Return,Cumulative_CNXSMALLCAP_Return,Strategy_Drawdown,CNXSMALLCAP_Drawdown,Equal_Weighted_Drawdown,Equal_Weighted_Drawdown_Cost_Adjusted
0,2015-02,158198.86,15.0,10.0,-25059.10,47405.02,17159.38,-19029.65,0.67,3.04,...,0.1065,0.1582,0.1065,0.04,0.1582,0.0400,0.0000,0.0000,0.0000,0.0000
1,2015-09,39054.42,7.0,5.0,-20778.78,16447.82,10108.19,-20778.78,0.86,2.88,...,0.1632,0.2035,0.1632,0.01,0.2035,0.0504,0.0000,0.0000,0.0000,0.0000
2,2015-10,-30336.52,7.0,4.0,-11961.19,-1695.59,0.00,-2707.17,0.00,0.00,...,0.1572,0.1670,0.1572,0.02,0.1670,0.0714,-0.1794,0.0000,-0.0076,-0.0368
3,2015-11,17437.94,22.0,9.0,-19862.55,25477.53,8202.92,-3192.85,0.36,1.27,...,0.2644,0.1873,0.2644,0.03,0.1873,0.1036,-0.0796,0.0000,0.0000,0.0000
4,2015-12,-10610.16,24.0,8.0,-20697.01,30889.20,15649.73,-4792.91,0.29,0.90,...,0.2871,0.1747,0.2871,0.02,0.1747,0.1256,-0.1415,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2024-07,-131444.66,23.0,9.0,-14864.83,1833.11,1833.11,-4798.20,0.04,0.01,...,14.1243,1.2918,14.1243,0.04,1.2918,2.9597,-0.2857,0.0000,0.0000,0.0000
108,2024-08,-43788.89,20.0,7.0,-8161.22,14938.99,14857.34,-3675.65,0.10,0.40,...,13.6660,1.1914,13.6660,0.01,1.1914,2.9993,-0.3412,0.0000,-0.0275,-0.0324
109,2024-09,-36017.82,8.0,4.0,-12593.02,-1325.08,0.00,-2769.60,0.00,0.00,...,13.6895,1.1125,13.6895,-0.01,1.1125,2.9593,-0.3848,-0.0133,-0.0218,-0.0308
110,2024-10,-16648.12,13.0,6.0,-8907.77,20464.38,5809.79,-5151.71,0.23,0.64,...,13.0241,1.0774,13.0241,-0.03,1.0774,2.8405,-0.4042,-0.0529,-0.0641,-0.0779


In [ ]:
# import requests
# import logging
# from datetime import datetime
# import pandas as pd
# import pytz

# # Define the timezone
# IST = pytz.timezone('Asia/Kolkata')

# # Set up logging
# def setup_logger():
#     logger = logging.getLogger("UpstoxAPITestLogger")
#     logger.setLevel(logging.INFO)

#     # Clear existing handlers
#     if logger.hasHandlers():
#         logger.handlers.clear()

#     # Console handler
#     console_handler = logging.StreamHandler()
#     formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
#     console_handler.setFormatter(formatter)
#     logger.addHandler(console_handler)

#     return logger

# logger = setup_logger()

# # Function to fetch Upstox historical candle data
# def fetch_upstox_data(scrip, instrument_key, from_date, to_date, interval="1minute"):
#     """
#     Fetch historical candle data from Upstox API.

#     Args:
#         scrip (str): Trading symbol of the scrip (e.g., "NSE:AUROPHARMA").
#         instrument_key (str): Instrument key from the mapping file.
#         from_date (str): Start date in 'YYYY-MM-DD' format.
#         to_date (str): End date in 'YYYY-MM-DD' format.
#         interval (str): Candle interval (e.g., "1minute", "day").

#     Returns:
#         pd.DataFrame or None: DataFrame of OHLCV data if successful, None otherwise.
#     """
#     logger.info(f"Fetching {interval} data for {scrip} ({instrument_key}) from {from_date} to {to_date}...")
#     base_url = "https://api.upstox.com/v2/historical-candle"
#     url = f"{base_url}/{instrument_key}/{interval}/{to_date}/{from_date}"

#     headers = {
#         'Accept': 'application/json'
#     }

#     try:
#         response = requests.get(url, headers=headers)

#         if response.status_code != 200:
#             logger.warning(f"HTTP {response.status_code} - {response.text}")
#             return None

#         data = response.json()

#         if 'data' not in data or 'candles' not in data['data']:
#             logger.warning(f"No candle data available for {scrip}.")
#             return None

#         # Extract candle data
#         candles = data['data']['candles']

#         # Convert to DataFrame
#         df = pd.DataFrame(candles, columns=['datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])
#         df['datetime'] = pd.to_datetime(df['datetime'])
#         df.set_index('datetime', inplace=True)

#         logger.info(f"Data successfully fetched for {scrip}. Total rows: {len(df)}")
#         return df

#     except requests.exceptions.RequestException as e:
#         logger.error(f"RequestException while fetching data for {scrip}: {e}")
#         return None
#     except Exception as e:
#         logger.error(f"Unexpected error while fetching data for {scrip}: {e}")
#         return None

# # Example usage
# if __name__ == "__main__":
#     # Example instrument mapping
#     instrument_mapping = {
#         "NSE:AUROPHARMA": "NSE_EQ|INE406A01037",
#         "NSE:APARINDS": "NSE_EQ|INE372A01015"  # Add other instruments as needed
#     }

#     # Test data
#     scrip = "NSE:AUROPHARMA"
#     instrument_key = instrument_mapping.get(scrip)
#     if not instrument_key:
#         logger.error(f"Instrument key not found for {scrip}.")
#     else:
#         # Specify the date range
#         from_date = "2023-05-01"
#         to_date = "2023-05-31"
#         interval = "1minute"

#         # Fetch data
#         df = fetch_upstox_data(scrip, instrument_key, from_date, to_date, interval)

#         # Process or save data
#         if df is not None:
#             output_file = f"{scrip.replace(':', '_')}_{from_date}_to_{to_date}.csv"
#             df.to_csv(output_file)
#             logger.info(f"Data saved to {output_file}")
#         else:
#             logger.warning(f"No data fetched for {scrip} during {from_date} to {to_date}.")
